In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
union_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W20` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W21`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W22` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W23` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W24` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W25` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W26` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W27` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W28` 
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W29`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W30`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W31`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10,
destination = 'scanning-database.PowerBI.PowerBI',
write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
) # set query limit to 10G
query_job = bigquery_client.query(union_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
data_in_use = rows.to_dataframe()

In [4]:
data_in_use.head()

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-05-28 11:07:00+00:00,2020-05-28 11:07:00+00:00,TBD,None,R4,None,None,KL18,None,Thursday Morning,Dummy,0,0.28,2020-23
1,2020-07-11 21:43:00+00:00,2020-07-11 22:26:00+00:00,Break,None,SE,move to other line,None,KL10,None,Saturday Afternoon,Dummy,0,43.10,2020-29
2,2020-07-11 14:33:00+00:00,2020-07-11 17:49:00+00:00,Break,None,A4,move to other line,None,KL2,None,Saturday Afternoon,Sanjay,0,196.37,2020-29
3,2020-06-07 07:38:00+00:00,2020-06-07 07:38:00+00:00,Production,None,K4,None,None,KL13,None,Sunday Morning,Tarun,0,0.13,2020-24
4,2020-06-06 10:45:00+00:00,2020-06-06 11:20:00+00:00,Break,None,E4,move to other line,None,KL4,None,Saturday Morning,Tarun,0,34.43,2020-24


In [5]:
data_in_use.shape

(85271, 14)